1. crim, уровень преступности на душу населения по городам.
2. zn, доля жилой земли, зонированной на участки площадью более 25 000 кв.футов.
3. indus, доля акров, не связанных с розничной торговлей, в расчете на город.
4. chas, фиктивная переменная реки Чарльз (= 1, если тракт граничит с рекой; 0 в противном случае).
5. nox, концентрация оксидов азота (частей на 10 миллионов).
6. rm, среднее количество комнат в доме.
7. age, доля квартир, занятых владельцами, построенных до 1940 года.
8. dis, средневзвешенное значение расстояний до пяти бостонских центров занятости.
9. rad, индекс доступности к радиальным магистралям.
10. tax, полная стоимость недвижимости - ставка налога за 10 000 долларов.
11. ptratio, соотношение учеников и учителей по городам.
12. black, 1000(Bk - 0,63)^2, где Bk - доля чернокожих по городам.
13. lstat, более низкий статус населения (в процентах).
14. medv, средняя стоимость домов, занятых владельцами, в 1000 долларов.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
df = pd.read_csv('boston_housing.csv')

In [3]:
df.sample(5,random_state=42)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
173,0.09178,0.0,4.05,0.0,0.510,6.416,84.1,2.6463,5.0,296.0,16.6,395.50,9.04,23.6
274,0.05644,40.0,6.41,1.0,0.447,6.758,32.9,4.0776,4.0,254.0,17.6,396.90,3.53,32.4
491,0.10574,0.0,27.74,0.0,0.609,5.983,98.8,1.8681,4.0,711.0,20.1,390.11,18.07,13.6
72,0.09164,0.0,10.81,0.0,0.413,6.065,7.8,5.2873,4.0,305.0,19.2,390.91,5.52,22.8
452,5.09017,0.0,18.10,0.0,0.713,6.297,91.8,2.3682,24.0,666.0,20.2,385.09,17.27,16.1


In [4]:
df.drop('MEDV',axis=1).head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [5]:
df['MEDV'].head()

0    24.0
1    21.6
2    34.7
3    33.4
4    36.2
Name: MEDV, dtype: float64

In [6]:
# разделим наш датасет на тренировочный и тестовый
X_train, X_test, y_train, y_test = train_test_split(df.drop('MEDV',axis=1), 
                                                    df['MEDV'], 
                                                    test_size=0.3, 
                                                    random_state=888)

In [7]:
print(X_train.shape)
print(X_test.shape)

(354, 13)
(152, 13)


In [8]:
# предскажем параметр MEDV с помощью линейной регрессии и посчитаем среднее значение RMSE
from sklearn.linear_model import LinearRegression

In [9]:
my_super_first_model = LinearRegression()

In [10]:
%%time
my_super_first_model.fit(X_train, y_train)

CPU times: total: 0 ns
Wall time: 34 ms


LinearRegression()

In [11]:
pred = my_super_first_model.predict(X_test)

1  Оценка на глаз для линейной регрессии

In [12]:
test_pred = X_test.copy()

In [13]:
test_pred['predict_Linear'] = pred
test_pred['real_MEDV'] = y_test

In [14]:
test_pred[['predict_Linear','real_MEDV']].head()

,predict_Linear,real_MEDV
212,22.332769,22.4
274,35.816878,32.4
12,20.622689,21.7
494,20.222810,24.5
363,20.528320,16.8


In [15]:
print('Ошибка RMSE c random seed 888 =', sqrt(mean_squared_error(y_test, pred)))

Ошибка RMSE c random seed 888 = 4.538174205735535


2  разные random seed для линейной регрессии

In [16]:
pr = []

for i in range(1000):
    X_train, X_test, y_train, y_test = train_test_split(df.drop('MEDV',axis=1), 
                                                    df['MEDV'], 
                                                    test_size=0.3, 
                                                    random_state=i)
    my_super_first_model = LinearRegression()
    my_super_first_model.fit(X_train, y_train)
    pred = my_super_first_model.predict(X_test)
#    print('Ошибка c random seed=',i,' RMSE =',mean_squared_error(y_test, pred))
    pr.append(sqrt(mean_squared_error(y_test, pred)))

print(f'С random seed от 0 до 1000 среднее значение RMSE для линейной регрессии = {pd.Series(pr).mean()}')    

С random seed от 0 до 1000 среднее значение RMSE для линейной регрессии = 4.910227422333465


In [17]:
# предскажем параметр MEDV с помощью деревьев решений и посчитаем среднее значение RMSE
from sklearn.tree import DecisionTreeRegressor

In [18]:
# разделим наш датасет на тренировочный и тестовый
X_train, X_test, y_train, y_test = train_test_split(df.drop('MEDV',axis=1), 
                                                    df['MEDV'], 
                                                    test_size=0.3, 
                                                    random_state=888)
my_super_second_model = DecisionTreeRegressor()
# %%time
my_super_second_model.fit(X_train, y_train)
pred2 = my_super_second_model.predict(X_test)

# оценка на глаз для деревьев решений
test_pred2 = X_test.copy()
test_pred2['predict_Tree'] = pred2
test_pred2['real_MEDV'] = y_test

In [19]:
# разные random seed для деревьев решений 
pr2 = []

for i in range(1000):
    X_train, X_test, y_train, y_test = train_test_split(df.drop('MEDV',axis=1), 
                                                    df['MEDV'], 
                                                    test_size=0.3, 
                                                    random_state=i)
    my_super_second_model = DecisionTreeRegressor()
    my_super_second_model.fit(X_train, y_train)
    pred2 = my_super_second_model.predict(X_test)
#    print('Ошибка c random seed=',i,' RMSE =',mean_squared_error(y_test, pred))
    pr2.append(sqrt(mean_squared_error(y_test, pred2)))

print(f'С random seed от 0 до 1000 среднее значение RMSE для деревьев решений  = {pd.Series(pr2).mean()}')    

С random seed от 0 до 1000 среднее значение RMSE для деревьев решений  = 4.709715636446686


In [20]:
# предскажем параметр MEDV с помощью метода опорных векторов и посчитаем среднее значение RMSE
from sklearn import svm

In [21]:
# разделим наш датасет на тренировочный и тестовый
X_train, X_test, y_train, y_test = train_test_split(df.drop('MEDV',axis=1), 
                                                    df['MEDV'], 
                                                    test_size=0.3, 
                                                    random_state=888)
my_super_third_model = svm.SVR()
# %%time
my_super_third_model.fit(X_train, y_train)
pred3 = my_super_third_model.predict(X_test)

# оценка на глаз для метода опорных векторов
test_pred3 = X_test.copy()
test_pred3['predict_SVM'] = pred3
test_pred3['real_MEDV'] = y_test

In [22]:
# разные random seed для метода опорных векторов
pr3 = []

for i in range(1000):
    X_train, X_test, y_train, y_test = train_test_split(df.drop('MEDV',axis=1), 
                                                    df['MEDV'], 
                                                    test_size=0.3, 
                                                    random_state=i)
    my_super_third_model = svm.SVR()
    my_super_third_model.fit(X_train, y_train)
    pred3 = my_super_third_model.predict(X_test)
#    print('Ошибка c random seed=',i,' RMSE =',mean_squared_error(y_test, pred))
    pr3.append(sqrt(mean_squared_error(y_test, pred3)))

print(f'С random seed от 0 до 1000 среднее значение RMSE для метода опорных векторов = {pd.Series(pr3).mean()}')   

С random seed от 0 до 1000 среднее значение RMSE для метода опорных векторов = 8.235661031565723


In [23]:
# выведем на экран результаты предсказания с random seed 888 для оценки на глаз
result = pd.concat([test_pred[['predict_Linear']], test_pred2[['predict_Tree']], test_pred3[['predict_SVM','real_MEDV']]], axis=1, join="inner")
result

,predict_Linear,predict_Tree,predict_SVM,real_MEDV
212,22.332769,22.5,23.108047,22.4
274,35.816878,33.1,24.295160,32.4
12,20.622689,22.5,22.837188,21.7
494,20.222810,20.6,21.102139,24.5
363,20.528320,19.9,15.217571,16.8
...,...,...,...,...
237,33.649605,31.7,22.387495,31.5
13,20.432405,20.4,22.547433,20.4
159,26.672551,24.4,19.691633,23.3
318,24.633583,17.5,22.492705,23.1


In [24]:
# среднее значение RMSE c random seed от 0 до 1000
print(f'Cреднее значение RMSE для линейной регресии: {pd.Series(pr).mean()}')
print(f'Cреднее значение RMSE для деревьев решений: {pd.Series(pr2).mean()}')
print(f'Cреднее значение RMSE для метода опорных векторов: {pd.Series(pr3).mean()}')

Cреднее значение RMSE для линейной регресии: 4.910227422333465
Cреднее значение RMSE для деревьев решений: 4.709715636446686
Cреднее значение RMSE для метода опорных векторов: 8.235661031565723


Значение RMSE из orange3:

линейная регрессия - 4.714

дерево решений - 3.928

метод опорных векторов - 5.928

Вывод.
Для существующего датасета с помощью библиотек sklearn и в orange3 получен примерно одинаковый результат:
RMSE имеет наименьшее значение для деревьев решений и максимальное для метода опорных векторов.